# Table of Contents

-  [About the dataset](#about)<br>
-  [Load the data](#load_data)<br>

# Imports

In [1]:
# for package auto reload
%load_ext autoreload
%autoreload 2

# for better rendering of plots in jupyter notebook
%matplotlib inline

In [2]:
# base modules
import os
import sys
import copy
import logging
from collections import OrderedDict

# for manipulating data
import numpy as np
import pandas as pd
import math
import dill

# for Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance
from scipy.cluster import hierarchy

# for visualization
from IPython.display import display
from matplotlib import pyplot as plt
import graphviz
import streamlit as st
# plotly
# seaborn
# altair

# custom module
from emlyon.utils import *


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [3]:
# path to repo
path_to_repo = os.getcwd()
path_to_repo

'/Users/nicolas/Desktop/Etudes/3_emlyon/2021:2022/Machine_Learning/project'

# Introduction: Crops Yield Project

### Project Ideas

Several ideas might pop up looking at this dataset, for example:

**Exploratory Data Analysis**
- Vizualisation of certain crops yields' temporal evolution per country
- Correlation between production & population

**Machine Learning**
- C02 emissions of crops using regression
- Clustering on CO2 crop emissioners
- Crop yields forecasting through time-series analysis using production & pop correlation

# Datasets Overview

## 1. Agricultural Crop Production

### Summary

Crop statistics for 173 products in Africa, America, Asia, Europe and Oceania, collected from 1961 to 2019.

### Description

Data from the Food and Agriculture Organization of the United Nations (FAO)

Achieving food security for all is at the heart of FAO's efforts - ensuring that people have regular access to enough quality food to lead active and healthy lives. Our three main objectives are: the eradication of hunger, food insecurity and malnutrition; the eradication of poverty and the promotion of economic and social progress for all; and the sustainable management and use of natural resources, including land, water, air, climate and genetic resources, for the benefit of present and future generations.

Primary crops, fibre crops. Crop statistics are recorded for 173 commodities, covering the following categories: Primary crops, Primary fibre crops, Cereals, Secondary cereals, Citrus, Fruit, Jute and related fibres, Oilcake equivalent, Primary oilseeds, Dry vegetables, Roots and tubers, Green fruits and vegetables and Melons. Data are expressed in terms of area harvested, quantity produced, yield and quantity of seed. The aim is to provide comprehensive coverage of production of all primary crops for all countries and regions of the world.

Source : Organisation des Nations Unies pour l'alimentation et l'agriculture (FAO)

## 2. Gas emissions Statistics

### Summary

The FAOSTAT domain Emissions Totals disseminates information estimates of CH4, N2O and CO2 emissions/removals and their aggregates in CO2eq in units of kilotonnes (kt, or 106 kg). 

### Description

The FAOSTAT domain Emissions Totals summarizes the greenhouse gas (GHG) emissions disseminated in the FAOSTAT Climate Change Emissions domains, generated from agriculture and forest land. They consist of methane (CH4), nitrous oxide (N2O) and carbon dioxide (CO2) emissions from crop and livestock activities, forest management and include land use and land use change processes. Data are computed at Tier 1 of the IPCC Guidelines for National greenhouse gas (GHG) Inventories (IPCC, 1996; 1997; 2000; 2002; 2006; 2014). Estimates are available by country, with global coverage for the period 1961–2019 with projections for 2030 and 2050 for some categories of emissions or 1990–2019 for others. The database is updated annually.

## 3. Population per country

The FAOSTAT Population module contains time series data on population, by sex and urban/rural. The series consist of both estimates and projections for different periods as available from the original sources, namely:
1. Population data refers to the World Population Prospects: The 2019 Revision from the UN Population Division.
2. Urban/rural population data refers to the World Urbanization Prospects: The 2018 Revision from the UN Population Division.

# Data Loading

## 1. Load the *Agricultural Crop data*

### Get a glimpse of one of the main datasets

In [4]:
# Loading Africa's dataset

url = 'https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/agriculture-crop-production/Production_Crops_E_Africa.csv'
crops_africa = pd.read_csv(url, sep=',',encoding='latin-1')
crops_africa.head(10)

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F
0,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,F,13300.0,...,40403.0,NaN,49983.0,NaN,50100.0,NaN,43043.0,NaN,35380.0,NaN
1,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,Fc,4511.0,...,18930.0,Fc,13223.0,Fc,12362.0,Fc,13292.0,Fc,20467.0,Fc
2,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,NaN,6000.0,...,76482.0,NaN,66095.0,NaN,61934.0,NaN,57213.0,NaN,72412.0,NaN
3,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,F,3100.0,...,41011.0,NaN,46070.0,NaN,44620.0,NaN,39034.0,NaN,32989.0,NaN
4,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,Fc,45161.0,...,110086.0,Fc,108716.0,Fc,110766.0,Fc,124970.0,Fc,169399.0,Fc
5,4,Algeria,515,Apples,5510,Production,tonnes,15400.0,NaN,14000.0,...,451472.0,NaN,500855.0,NaN,494239.0,NaN,487808.0,NaN,558830.0,NaN
6,4,Algeria,526,Apricots,5312,Area harvested,ha,4200.0,F,4600.0,...,38857.0,NaN,38239.0,NaN,44307.0,NaN,35500.0,NaN,30861.0,NaN
7,4,Algeria,526,Apricots,5419,Yield,hg/ha,30286.0,Fc,30000.0,...,75530.0,Fc,67149.0,Fc,57980.0,Fc,68237.0,Fc,67789.0,Fc
8,4,Algeria,526,Apricots,5510,Production,tonnes,12720.0,NaN,13800.0,...,293486.0,NaN,256771.0,NaN,256890.0,NaN,242243.0,NaN,209204.0,NaN
9,4,Algeria,366,Artichokes,5312,Area harvested,ha,5000.0,F,5000.0,...,4674.0,NaN,5174.0,NaN,5532.0,NaN,5784.0,NaN,5792.0,NaN


In [5]:
crops_africa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Columns: 125 entries, Area Code to Y2019F
dtypes: float64(59), int64(3), object(63)
memory usage: 8.7+ MB


### Load all datasets

In [6]:
def open_all_datasets_df(item):
    df = pd.read_csv('https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/agriculture-crop-production/Production_Crops_E_' + str(item) + '.csv', low_memory = False, encoding='latin1')
    df['continent'] = item
    return df

In [7]:
countries = ['Africa','Americas','Asia','Europe','Oceania']

In [8]:
crops_raw = pd.DataFrame()

for country in countries:
    crops_raw = crops_raw.append(open_all_datasets_df(country))

/var/folders/r7/nptln7p562l2yhwpxw1nxp880000gn/T/ipykernel_71115/1436319694.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crops_raw = crops_raw.append(open_all_datasets_df(country))
/var/folders/r7/nptln7p562l2yhwpxw1nxp880000gn/T/ipykernel_71115/1436319694.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crops_raw = crops_raw.append(open_all_datasets_df(country))
/var/folders/r7/nptln7p562l2yhwpxw1nxp880000gn/T/ipykernel_71115/1436319694.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crops_raw = crops_raw.append(open_all_datasets_df(country))
/var/folders/r7/nptln7p562l2yhwpxw1nxp880000gn/T/ipykernel_71115/1436319694.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

In [9]:
crops_raw

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2015F,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,continent
0,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,F,13300.0,...,NaN,49983.0,NaN,50100.0,NaN,43043.0,NaN,35380.0,NaN,Africa
1,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,Fc,4511.0,...,Fc,13223.0,Fc,12362.0,Fc,13292.0,Fc,20467.0,Fc,Africa
2,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,NaN,6000.0,...,NaN,66095.0,NaN,61934.0,NaN,57213.0,NaN,72412.0,NaN,Africa
3,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,F,3100.0,...,NaN,46070.0,NaN,44620.0,NaN,39034.0,NaN,32989.0,NaN,Africa
4,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,Fc,45161.0,...,Fc,108716.0,Fc,110766.0,Fc,124970.0,Fc,169399.0,Fc,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,155,Vanuatu,1720,"Roots and Tubers, Total",5419,Yield,hg/ha,225455.0,Fc,227273.0,...,Fc,82306.0,Fc,82341.0,Fc,82374.0,Fc,82407.0,Fc,Oceania
1650,155,Vanuatu,1720,"Roots and Tubers, Total",5510,Production,tonnes,24800.0,A,25000.0,...,A,52314.0,A,52838.0,A,53362.0,A,53886.0,A,Oceania
1651,155,Vanuatu,1735,Vegetables Primary,5312,Area harvested,ha,200.0,A,210.0,...,A,804.0,A,810.0,A,817.0,A,824.0,A,Oceania
1652,155,Vanuatu,1735,Vegetables Primary,5419,Yield,hg/ha,150000.0,Fc,150000.0,...,Fc,165547.0,Fc,166358.0,Fc,166952.0,Fc,167524.0,Fc,Oceania


It appears that columns representing the data per year come with an indicator "F", let's dig into it !

In [10]:
flags_df = pd.read_csv("https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/agriculture-crop-production/flags.csv", encoding="latin1")
flags_df

,"ï»¿""Flag""",Flags
0,NaN,Official data
1,*,Unofficial figure
2,A,"Aggregate, may include official, semi-official..."
3,Bk,Break in series
4,C,Calculated
5,Ce,Calculated data based on estimated data
6,Cv,Calculated through value
7,E,Expert sources from FAO (including other divis...
8,F,FAO estimate
9,Fb,Data obtained as a balance


Each year column comes with another column, stating the source of the figures for this year (whether the figure was calculated with official data, estimated, etc).
It is good info to know, but let's put these data pieces aside and keep a smaller df:

In [90]:
crops_all = copy.deepcopy(crops_raw)
crops_all = crops_all.loc[:, ~crops_all.columns.str.endswith('F')]
crops_all = crops_all[crops_all.columns[~crops_all.columns.str.endswith('F')]]
first_col = crops_all.pop('continent')
crops_all.insert(0, 'Continent', first_col)
crops_all

,Continent,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,Africa,4,Algeria,221,"Almonds, with shell",5312,Area harvested,ha,13300.0,13300.0,...,54485.0,52245.0,49975.0,49011.0,39050.0,40403.0,49983.0,50100.0,43043.0,35380.0
1,Africa,4,Algeria,221,"Almonds, with shell",5419,Yield,hg/ha,4511.0,4511.0,...,10457.0,9689.0,13304.0,12965.0,16601.0,18930.0,13223.0,12362.0,13292.0,20467.0
2,Africa,4,Algeria,221,"Almonds, with shell",5510,Production,tonnes,6000.0,6000.0,...,56973.0,50621.0,66487.0,63545.0,64827.0,76482.0,66095.0,61934.0,57213.0,72412.0
3,Africa,4,Algeria,515,Apples,5312,Area harvested,ha,3400.0,3100.0,...,52419.0,51080.0,48828.0,48064.0,40418.0,41011.0,46070.0,44620.0,39034.0,32989.0
4,Africa,4,Algeria,515,Apples,5419,Yield,hg/ha,45294.0,45161.0,...,72233.0,79112.0,81414.0,94860.0,114507.0,110086.0,108716.0,110766.0,124970.0,169399.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,Oceania,155,Vanuatu,1720,"Roots and Tubers, Total",5419,Yield,hg/ha,225455.0,227273.0,...,82072.0,80645.0,82258.0,82540.0,82518.0,82467.0,82306.0,82341.0,82374.0,82407.0
1650,Oceania,155,Vanuatu,1720,"Roots and Tubers, Total",5510,Production,tonnes,24800.0,25000.0,...,47700.0,50000.0,51000.0,52000.0,52374.0,52251.0,52314.0,52838.0,53362.0,53886.0
1651,Oceania,155,Vanuatu,1735,Vegetables Primary,5312,Area harvested,ha,200.0,210.0,...,753.0,762.0,750.0,779.0,791.0,801.0,804.0,810.0,817.0,824.0
1652,Oceania,155,Vanuatu,1735,Vegetables Primary,5419,Yield,hg/ha,150000.0,150000.0,...,160651.0,161444.0,166667.0,163248.0,163540.0,164819.0,165547.0,166358.0,166952.0,167524.0


## 2. Preprocess the *Gas Emissions data*

In [39]:
# Loading Gas Emissions' dataset

url = 'https://raw.githubusercontent.com/nicoboou/ml_eml/main/data/emissions/emissions_full.csv'
emissions_df = pd.read_csv(url, sep=',',encoding='latin-1',low_memory=False)
emissions_df.head(10)

,Code zone,Zone,Code Produit,Produit,Code Élément,Élément,Code source,Source,Unité,Y1961,...,Y2019N,Y2020,Y2020F,Y2020N,Y2030,Y2030F,Y2030N,Y2050,Y2050F,Y2050N
0,2,Afghanistan,5058,Fermentation entérique,7225,Émissions (CH4),3050,FAO TIER 1,kilotonnes,240.6831,...,NaN,NaN,NaN,NaN,453.7474,Fc,NaN,603.6185,Fc,NaN
1,2,Afghanistan,5058,Fermentation entérique,7225,Émissions (CH4),3051,UNFCCC,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,5058,Fermentation entérique,724413,Émissions (CO2eq) venant de CH4 (AR5),3050,FAO TIER 1,kilotonnes,6739.1279,...,NaN,NaN,NaN,NaN,12704.9283,Fc,NaN,16901.3173,Fc,NaN
3,2,Afghanistan,5058,Fermentation entérique,724413,Émissions (CO2eq) venant de CH4 (AR5),3051,UNFCCC,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Afghanistan,5058,Fermentation entérique,723113,Émissions (CO2eq) (AR5),3050,FAO TIER 1,kilotonnes,6739.1279,...,NaN,NaN,NaN,NaN,12704.9283,Fc,NaN,16901.3173,Fc,NaN
5,2,Afghanistan,5058,Fermentation entérique,723113,Émissions (CO2eq) (AR5),3051,UNFCCC,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,Afghanistan,5059,Gestion du fumier,7225,Émissions (CH4),3050,FAO TIER 1,kilotonnes,11.6228,...,NaN,NaN,NaN,NaN,27.2114,Fc,NaN,35.2700,Fc,NaN
7,2,Afghanistan,5059,Gestion du fumier,7225,Émissions (CH4),3051,UNFCCC,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,Afghanistan,5059,Gestion du fumier,7230,Émissions (N2O),3050,FAO TIER 1,kilotonnes,0.3992,...,NaN,NaN,NaN,NaN,0.5770,Fc,NaN,0.8470,Fc,NaN
9,2,Afghanistan,5059,Gestion du fumier,724413,Émissions (CO2eq) venant de CH4 (AR5),3050,FAO TIER 1,kilotonnes,325.4372,...,NaN,NaN,NaN,NaN,761.9188,Fc,NaN,987.5598,Fc,NaN


**Let's focus for now on CO2 emissions**

In [116]:
co2_df = emissions_df.drop(emissions_df[emissions_df['Code Élément'] != 7273].index)
co2_df

,Code zone,Zone,Code Produit,Produit,Code Élément,Élément,Code source,Source,Unité,Y1961,...,Y2019N,Y2020,Y2020F,Y2020N,Y2030,Y2030F,Y2030N,Y2050,Y2050F,Y2050N
47,2,Afghanistan,6750,Conversion nette de forêt,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2,Afghanistan,6751,Terres forestières,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,2,Afghanistan,6993,Feux de tourbières,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,2,Afghanistan,6994,On-farm energy use,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2,Afghanistan,1707,UTCATF,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35722,5873,OECD,1707,UTCATF,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35730,5873,OECD,6824,AFOLU,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35738,5873,OECD,6995,Émissions sur les terres agricoles,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35746,5873,OECD,6996,Farm-gate emissions,7273,Émissions (CO2),3050,FAO TIER 1,kilotonnes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
### MEMO: calculer dans un nouveau df le total d'émissions de CO2 par année

# Exploratory Data Analysis

Before trying to preprocess the data in order to make use of it for our machine learning tasks, let's explore what we have & try to figure out some primary findings.

## 1. Exploring the Agricultural Crop data

In [121]:
crops_all.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38146 entries, 0 to 1653
Data columns (total 67 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Continent     38146 non-null  object 
 1   Area Code     38146 non-null  int64  
 2   Area          38146 non-null  object 
 3   Item Code     38146 non-null  int64  
 4   Item          38146 non-null  object 
 5   Element Code  38146 non-null  int64  
 6   Element       38146 non-null  object 
 7   Unit          38146 non-null  object 
 8   Y1961         22639 non-null  float64
 9   Y1962         22658 non-null  float64
 10  Y1963         22656 non-null  float64
 11  Y1964         22683 non-null  float64
 12  Y1965         22683 non-null  float64
 13  Y1966         22780 non-null  float64
 14  Y1967         22815 non-null  float64
 15  Y1968         22878 non-null  float64
 16  Y1969         22911 non-null  float64
 17  Y1970         22997 non-null  float64
 18  Y1971         23070 non-nul

/var/folders/r7/nptln7p562l2yhwpxw1nxp880000gn/T/ipykernel_71115/906973716.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  crops_all.info(verbose=True, null_counts=True)


In [13]:
crops_all.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Area Code,38146.0,NaN,NaN,NaN,130.297069,75.053504,1.0,63.0,126.0,196.0,299.0
Area,38146,210,"China, mainland",398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item Code,38146.0,NaN,NaN,NaN,614.361663,547.910943,15.0,236.0,446.0,656.0,1841.0
Item,38146,175,"Roots and Tubers, Total",618,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Element Code,38146.0,NaN,NaN,NaN,5414.258349,82.044592,5312.0,5312.0,5419.0,5510.0,5510.0
...,...,...,...,...,...,...,...,...,...,...,...
Y2016,33042.0,NaN,NaN,NaN,722969.65786,10249828.135251,0.0,3191.25,22985.0,123213.75,768594154.0
Y2017,33100.0,NaN,NaN,NaN,738894.115952,10171664.733574,0.0,3191.0,22932.0,124309.25,758646205.0
Y2018,33392.0,NaN,NaN,NaN,734436.155127,10245919.307647,0.0,2476.25,21138.0,121818.75,747060316.0
Y2019,33437.0,NaN,NaN,NaN,742458.525197,10346386.885358,0.0,2470.0,21430.0,121779.0,752895389.0


In [12]:
crops_all["Element"].unique()

array(['Area harvested', 'Yield', 'Production'], dtype=object)

It appears that we have data on 3 main elements thanks to this dataset:
- Area harvested
- Yield
- Production

**Let's inspect missing values**

In [ ]:
crops_all.isnull().describe().T

In [ ]:
df_raw.isnull().sum().sort_index()/len(df_raw)

**What is the distribution of our main numerical variables ?**

In [ ]:
# Let's have a look at the distribution of the numeric variables 

#crops_main_num_var_df.hist(bins=100, figsize=(20,15))
#plt.show()

**Let's have a lot at some first correlations**

In [ ]:
# Let's have a look at some first correlations 

#corr_matrix = crops_main_num_var_df.corr()
#corr_matrix

In [123]:
#corr_matrix = players_22_stats.corr()
#matrix = np.triu(corr_matrix)
#sns.heatmap(corr_matrix, annot=True, vmin = -1, vmax = 1, mask=matrix)

First insights:

**Let's now have a look at the scatter matrix, which is a statistic that is used to make estimates of the covariance matrix, for instance of the multivariate normal distribution.**

In [122]:
#from pandas.plotting import scatter_matrix

#scatter_matrix(df, figsize=(12, 8))
#plt.show()

**What could be interesting is to see the share of production of Wheat per country**

# Machine Learning

## Regression Model

### Data Preprocessing

We choose to preprocess the data according to the use we will make of it. Thus we will preprocess the data in order to have **numerical, approximately symmetrically distributed data** for our **regression** model.

**Some outliers ? Let's make a non-linear re-expression of the dependent variable using the logarithm.**

In [118]:
# Use of the log function

#We first keep the values inside a list
#list_to_build = df['column'].tolist()

#We then apply the log function on it
#df['column'] = np.log(df['column'])

**Some categorical variables ? Let's encode them into numerical ones to be able to use them in our regression model, and also fill the missing values with the median for each corresponding feature**

In [119]:
#df['column'].astype('category').cat.codes
#df, y, nas = proc_df(df_raw, 'target_variable')

**Save the pre-processed data into a *feather* file**

In [ ]:
path_to_tmp = os.path.join(path_to_repo, "data", "tmp")

In [ ]:
path_to_tmp

In [ ]:
os.makedirs(path_to_tmp, exist_ok = True)

In [ ]:
path_to_bulldozers_raw = os.path.join(path_to_tmp, 'bulldozers-raw')

In [ ]:
df_raw.to_feather(path_to_bulldozers_raw)

## Classification Model

### Data Preprocessing

## Clustering Model

### Data Preprocessing

## Time-series Analysis

### Data Preprocessing

In order to be able to make any use of our data in a time series analysis purpose, we will transform our datasets so that we have **dates** as index.

In [114]:
df = crops_all.set_index(list(crops_all.columns[:7]))
#df = pd.MultiIndex.from_frame(df)
#df = df.to_frame()
df.T.loc[['Y1986']]

Continent                 Africa                                             \
Area Code                    4                                                
Area                     Algeria                                              
Item Code                   221                               515             
Item         Almonds, with shell                            Apples            
Element Code                5312    5419       5510           5312     5419   
Element           Area harvested   Yield Production Area harvested    Yield   
Y1986                    20860.0  3864.0     8060.0         8000.0  50874.0   

Continent                                                                  \
Area Code                                                                   
Area                                                                        
Item Code                         526                                366    
Item                          Apricots                         Artichokes   
Element Code       5510           5312     5419       5510           5312   
Element      Production Area harvested    Yield Production Area harvested   
Y1986           40699.0         9300.0  39820.0    37033.0          830.0   

Continent     ...                   Oceania                                   \
Area Code     ...                       155                                    
Area          ...                   Vanuatu                                    
Item Code     ...                      1841                     1732           
Item          ... Oilcrops, Cake Equivalent Oilcrops, Oil Equivalent           
Element Code  ...                      5510                     5312    5419   
Element       ...                Production           Area harvested   Yield   
Y1986         ...                   28140.0                  70850.0  6373.0   

Continent                                                             \
Area Code                                                              
Area                                                                   
Item Code                                  1720                        
Item                    Roots and Tubers, Total                        
Element Code       5510                    5312      5419       5510   
Element      Production          Area harvested     Yield Production   
Y1986           45156.0                  3500.0  111429.0    39000.0   

Continent                                             
Area Code                                             
Area                                                  
Item Code                  1735                       
Item         Vegetables Primary                       
Element Code               5312      5419       5510  
Element          Area harvested     Yield Production  
Y1986                     485.0  156701.0     7600.0  

[1 rows x 38146 columns]

In [115]:
### MEMO: trop complexe, focus simplement sur le blé + enlever les codes

### Analysis

**1. Worldwide Production of Wheat**

*Moving Average*

*Exponential Weighted Function*

*ARIMA*

**2 .Correlation between CO2 Emissions & Crops Productions**

### CO2 Emissions Trend Prediction

1. Analyse des *feature_importances_* d'une regression linéraire sur les prédictions établies par la FAO sur les émissions de CO2

2. Utilisation de ces informations pour construire une time-series analysis et prédire une trend à horizon 2050

3. Comparer ces résultats avec les estimations de la FAO

### Crops Yield Estimations

1. En fonction de l'augmentation de la population, prédire les rendements nécessaires